# CRF entity recognition evaluation

In [1]:
from IPython.core.display import Markdown, HTML, display

import sys
sys.path.insert(0, '..')  # noqa
import settings  # noqa

import crf_data_generator as cdg
import pycrfsuite

import numpy as np

In [2]:
#data = cdg.ConlluCRFReader("../" + settings.gzipped_conllu_data_root + "recipes2.conllu.gz")
data = cdg.ConlluCRFReader("filtered_recipes.conllu")

data_iterator = iter(data)

In [3]:
def sentence_as_markdown_table( tokens, labels = None, predictions = None):
    n = len(tokens)
    s = "<table>\n<tr>\n<th>Sentence:</th>\n"
    for t in tokens:
        s += f"<th>{t}</th>"
    
    s += "<tr>\n"
    
    if labels is not None:
        s += "<th>labels:</th>" + "".join([f"<th>{l}</th>" for l in labels])
        s += "</tr>\n"
    
    if predictions is not None:
        s+= "<th>Predicitions:</th>" + "".join([f"<th>{p}</th>" for p in predictions])
    
    display(HTML(s + "</tr>\n</table>\n\n"))

    
    

In [4]:
tagger = pycrfsuite.Tagger()
tagger.open('test.crfsuite')

## Error Rate:

In [5]:
label_actions = []
label_ingredients = []
pred_actions = []
pred_ingredients = []

for features, labels, tokens in data:
    pred = tagger.tag(features)
    for i in range(len(tokens)):
        label_actions.append(labels[i] == "action")
        label_ingredients.append(labels[i] == "ingredient")
        pred_actions.append(pred[i] == "action")
        pred_ingredients.append(pred[i] == "ingredient")
    

IndexError: list index out of range

In [ ]:
label_actions = np.array(label_actions, dtype=np.bool)
label_ingredients = np.array(label_ingredients, dtype=np.bool)
pred_actions = np.array(pred_actions, dtype=np.bool)
pred_ingredients = np.array(pred_ingredients, dtype=np.bool)

len(label_actions)

In [ ]:
a_tp = np.sum(label_actions & pred_actions)
i_tp = np.sum(label_ingredients & pred_ingredients)

a_fp = np.sum(np.logical_not(label_actions) & pred_actions)
i_fp = np.sum(np.logical_not(label_ingredients) & pred_ingredients)

Markdown(f"""
* **\# all tokens**: {len(label_ingredients)}


* **\# real actions**: {np.sum(label_actions)}
* **\# predicted actions**: {np.sum(pred_actions)}


* **\# real ingredients**: {np.sum(label_ingredients)}
* **\# predicted ingredients**: {np.sum(pred_ingredients)} 


* **action error**: {1 - np.sum(label_actions == pred_actions) / len(label_ingredients)}
* **ingredient error**: {1 - np.sum(label_ingredients == pred_ingredients) / len(label_ingredients)}

* **action true positives**: {a_tp} of {np.sum(label_actions)} ({a_tp / np.sum(label_actions)})
* **ingredient true positives**: {i_tp} of {np.sum(label_ingredients)} ({i_tp / np.sum(label_ingredients)})

* **action false positives**: {a_fp} of {np.sum(label_actions)} ({a_fp / np.sum(label_actions)})
* **ingredient false positives**: {i_fp} of {np.sum(label_ingredients)} ({i_fp / np.sum(label_ingredients)})



""")

## Evaluation Example

In [9]:
n = 0

for i, (features, labels, tokens) in enumerate(data):
    if i > 100:
        break
    
    prediction = tagger.tag(features)
    
    n += len(tokens)
    
    sentence_as_markdown_table(tokens, labels, prediction)

In [10]:
n

730

In [16]:
730 - 92 - 16-24

598

In [17]:
79/730

0.10821917808219178